# 🔧 Xenon: XML Repair for LLM-Generated XML

Welcome to the interactive Xenon demo! This notebook shows you how to repair malformed XML commonly generated by Large Language Models.

**What is Xenon?**
- Zero-dependency Python library
- Fixes truncated, malformed, and messy XML from LLMs
- Simple API with robust error handling

**Open in Colab**: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MarsZDF/xenon/blob/main/xenon_demo.ipynb)

## 📦 Installation

Run this cell to install Xenon (will be available via pip soon):

In [ ]:
# Install Xenon directly from GitHub
!pip install -q git+https://github.com/MarsZDF/xenon.git

from xenon import repair_xml, parse_xml, repair_xml_safe, repair_xml_lenient, ValidationError

print("✅ Xenon installed and imported successfully!")

---

# 🎯 The 4 Major LLM XML Failure Modes

Xenon handles the most common ways LLMs break XML:

## 1️⃣ Truncation / Cut-off

LLMs run out of tokens mid-tag and leave XML incomplete:

In [ ]:
# LLM output that got cut off
truncated = '<root><user name="alice"<address><city>San Francisco'

print("❌ BROKEN XML:")
print(truncated)
print()

# Xenon repairs it
repaired = repair_xml(truncated)
print("✅ REPAIRED XML:")
print(repaired)

## 2️⃣ Conversational Fluff

LLMs wrap XML in conversational text:

In [ ]:
# LLM adds unnecessary commentary
fluff = '''
Sure! Here's the XML you requested:

<root>
    <message>Hello World</message>
    <status>success</status>
</root>

Hope this helps! Let me know if you need anything else.
'''

print("❌ MESSY OUTPUT:")
print(repr(fluff))
print()

# Xenon extracts just the XML
repaired = repair_xml(fluff)
print("✅ CLEAN XML:")
print(repaired)

## 3️⃣ Malformed Attributes

LLMs forget to quote attribute values:

In [ ]:
# Missing quotes around attributes
unquoted = '<product id=12345 category=electronics price=299.99 inStock=true>'

print("❌ INVALID ATTRIBUTES:")
print(unquoted)
print()

# Xenon adds quotes
repaired = repair_xml(unquoted)
print("✅ VALID ATTRIBUTES:")
print(repaired)

## 4️⃣ Unescaped Entities

LLMs forget to escape special characters like `&` and `<`:

In [ ]:
# Special characters not escaped
unescaped = '<description>Price: $5 < $10 & shipping included</description>'

print("❌ UNESCAPED ENTITIES:")
print(unescaped)
print()

# Xenon escapes them
repaired = repair_xml(unescaped)
print("✅ PROPERLY ESCAPED:")
print(repaired)

---

# 🔥 Combined Failure Modes

Real LLM outputs often have MULTIPLE issues at once:

In [ ]:
# Everything broken at once!
nightmare = '''
Here's the user data you asked for:

<users>
    <user id=1001 role=admin>
        <name>John Smith & Associates</name>
        <email>john@example.com</email>
        <status>Active & Verified
'''

print("❌ COMPLETE DISASTER:")
print(nightmare)
print()

# Xenon handles it all
repaired = repair_xml(nightmare)
print("✅ FULLY REPAIRED:")
print(repaired)
print()

# What Xenon fixed:
print("🔧 Xenon fixed:")
print("  • Removed conversational text")
print("  • Added quotes to attributes (id=1001 → id=\"1001\")")
print("  • Escaped & symbols (& → &amp;)")
print("  • Closed truncated tags (</status>, </user>, </users>)")

---

# 📊 Parse to Dictionary

Convert repaired XML to Python dictionaries for easy data extraction:

In [ ]:
# Malformed XML from LLM
malformed = '<response status=success><data count=3><item>Apple</item><item>Banana</item><item>Orange'

print("📝 Input:")
print(malformed)
print()

# Parse directly (repairs automatically)
data = parse_xml(malformed)

print("📦 Parsed Dictionary:")
import json
print(json.dumps(data, indent=2))
print()

# Access data easily
print("🎯 Extracted Values:")
print(f"  Status: {data['response']['@attributes']['status']}")
print(f"  Count: {data['response']['data']['@attributes']['count']}")
print(f"  Items: {data['response']['data']['item']}")

---

# 🛡️ Error Handling Modes

Xenon provides 3 modes for different use cases:

## Safe Mode (Production-Ready)

In [ ]:
# Safe mode validates input and provides helpful errors

print("1️⃣ Valid XML:")
result = repair_xml_safe('<root><item>test</item>')
print(f"   ✅ {result}")
print()

print("2️⃣ Invalid input (None):")
try:
    result = repair_xml_safe(None)
except ValidationError as e:
    print(f"   ❌ {e}")
print()

print("3️⃣ Empty string with allow_empty=True:")
result = repair_xml_safe('', allow_empty=True)
print(f"   ✅ Returns: {repr(result)}")
print()

print("4️⃣ Size limit protection:")
try:
    huge = '<root>' + 'x' * 10000
    result = repair_xml_safe(huge, max_size=5000)
except ValidationError as e:
    print(f"   ❌ {e}")

## Lenient Mode (Never Crashes)

In [ ]:
# Lenient mode NEVER raises exceptions

print("Testing lenient mode with various inputs:")
print()

test_cases = [
    (None, "None"),
    (123, "Integer"),
    ("", "Empty string"),
    (["<root>"], "List"),
    ("<root><item", "Truncated XML"),
]

for inp, description in test_cases:
    result = repair_xml_lenient(inp)
    print(f"  {description:20s} → {repr(result)[:40]}")

---

# 🎮 Interactive Playground

Try your own malformed XML below!

In [ ]:
# ✏️ Edit this XML and run the cell to see how Xenon repairs it!

your_xml = '''
Sure, here's the XML:

<config>
    <database host=localhost port=5432>
        <credentials user=admin password=secret123
'''

print("📥 YOUR INPUT:")
print(your_xml)
print()
print("=" * 60)
print()
print("📤 XENON OUTPUT:")
repaired = repair_xml(your_xml)
print(repaired)
print()
print("=" * 60)
print()
print("📊 AS DICTIONARY:")
data = parse_xml(your_xml)
print(json.dumps(data, indent=2))

---

# 🌍 Real-World LLM Examples

Examples from actual LLM outputs:

## Example 1: ChatGPT API Response

In [ ]:
chatgpt_output = '''
Here's the product catalog in XML format:

<catalog>
    <product id=A001 category=electronics>
        <name>Laptop Pro 15</name>
        <price currency=USD>1299.99</price>
        <description>High-performance laptop with 16GB RAM & SSD</description>
        <inStock>true</inStock>
    </product>
    <product id=A002 category=electronics>
        <name>Wireless Mouse</name>
        <price currency=USD>29.99
'''

print("Before:")
print(chatgpt_output)
print("\n" + "=" * 60 + "\n")
print("After:")
print(repair_xml(chatgpt_output))

## Example 2: Claude Code Generation

In [ ]:
claude_output = '''
I'll generate the XML configuration for you:

<configuration environment=production>
    <server>
        <host>api.example.com</host>
        <port>8080</port>
        <ssl enabled=true cert=/path/to/cert.pem>
    </server>
    <database type=postgresql>
        <connection string=postgresql://user@localhost:5432/db
'''

print("Before:")
print(claude_output)
print("\n" + "=" * 60 + "\n")
print("After:")
repaired = repair_xml(claude_output)
print(repaired)
print("\n" + "=" * 60 + "\n")
print("Parsed:")
print(json.dumps(parse_xml(claude_output), indent=2))

## Example 3: Structured Data Extraction

In [ ]:
# LLM extracting data from text into XML
extraction = '''
Based on the article, here's the extracted information:

<article>
    <title>Breaking News: Tech Company Valued at $5B < $10B
    <author name=Jane Doe role=senior-reporter>
    <published date=2024-01-15 time=14:30:00>
    <tags>
        <tag>technology</tag>
        <tag>business & finance</tag>
        <tag>startups
'''

print("Malformed extraction:")
print(extraction)
print("\n" + "=" * 60 + "\n")
print("Cleaned up:")
repaired = repair_xml(extraction)
print(repaired)
print("\n" + "=" * 60 + "\n")
print("Structured data:")
data = parse_xml(extraction)
print(json.dumps(data, indent=2))

---

# 🎓 Summary

## What Xenon Fixes:
- ✅ **Truncation**: Auto-closes open tags
- ✅ **Conversational Fluff**: Extracts pure XML
- ✅ **Malformed Attributes**: Adds missing quotes
- ✅ **Unescaped Entities**: Escapes `&` and `<`

## Three Modes:
- 🛡️ **Safe Mode**: Production-ready with validation
- 🚀 **Lenient Mode**: Never crashes, always returns something
- ⚡ **Default Mode**: Fast, assumes valid string input

## Installation:
```bash
pip install xenon  # Coming soon!
```

## Resources:
- **GitHub**: https://github.com/MarsZDF/xenon
- **Documentation**: See README.md
- **License**: MIT

---

### 💡 Have fun repairing XML! 🔧